In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd

data = pd.read_csv("../input/resume-dataset/UpdatedResumeDataSet.csv")

In [ ]:
data.head()

In [ ]:
# To identify the number of unique categories

print (data['Category'].unique())
print("")
print("Number of datapoints in each categories")
print(data['Category'].value_counts())

In [ ]:
#Visualization of various categories
data['Category'].value_counts(sort=True).nlargest(25).plot.bar()

In [ ]:
# # remove empty documents from the dataframe
# empty_doc = []
# for i in range(len(data)):
#   if(len(data['Resume'][i]) <=10):
#     empty_doc.append(i)
# print(empty_doc)
# data = data.drop(empty_doc)

In [ ]:
#We now need to clean the text in the "Resume" column. So some standard data preprocessing steps are followed as below.


import re
import string
import nltk
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')

def clean_text(text):
    #convert text to lowercase
    text = text.lower() 
    #remove any numeric characters
    text = ''.join([word for word in text if not word.isdigit()]) 

#     text = [word for word in text if re.search("\d", word)== None]
    # remove URLs
    text = re.sub('http\S+\s*', ' ', text)  
    # remove RT and cc
    text = re.sub('RT|cc', ' ', text)
    # remove hashtags
    text = re.sub('#\S+', '', text)  
    # remove mentions
    text = re.sub('@\S+', '  ', text)  
    #punctuations removal
    text = "".join([word for word in text if word not in string.punctuation])
    text = re.sub("\W", " ", str(text))
    #stopwords removal
    ext = [word for word in text.split() if word not in stopwords]
    #replace consecutive non-ASCII characters with a space
    text = re.sub(r'[^\x00-\x7f]',r' ', text) 
    #extra whitespace removal
    text = re.sub('\s+', ' ', text)
    return text

data['cleaned_text'] = data['Resume'].apply(lambda x: clean_text(x))

In [ ]:
data

In [ ]:
data["cleaned_text"][10]

In [ ]:
sent_lens = []
for i in data.cleaned_text:
    length = len(i.split())
    sent_lens.append(length)
    
print(len(sent_lens))
print(max(sent_lens))

In [ ]:
data["Resume"][100]

In [ ]:
data["cleaned_text"][100]

In [ ]:
from sklearn.preprocessing import LabelEncoder

labelencoder = LabelEncoder()
data["Category_N"] = labelencoder.fit_transform(data["Category"])
# print(type(labels))

data.tail()


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

Text = data['cleaned_text'].values
op_labels = data['Category_N'].values
word_vectorizer = TfidfVectorizer(max_features = 1500)
word_vectorizer.fit(Text)
features = word_vectorizer.transform(Text)

In [ ]:
word_vectorizer.get_feature_names()

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(features,op_labels,random_state=0, test_size=0.2)



In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics


clf = OneVsRestClassifier(KNeighborsClassifier())
clf.fit(X_train, y_train)
prediction = clf.predict(X_test)
print('Accuracy of KNeighbors Classifier on training set: {:.2f}'.format(clf.score(X_train, y_train)))
print('Accuracy of KNeighbors Classifier on test set: {:.2f}'.format(clf.score(X_test, y_test)))


In [ ]:
print("\n Classification report for classifier %s:\n%s\n" % (clf, metrics.classification_report(y_test, prediction)))
